<a href="https://colab.research.google.com/github/aKeles001/RNN_Large_Text_Generator/blob/main/RNN_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [ ]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [ ]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



# Encode and Decode

In [ ]:
vocab = sorted(set(text))

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [ ]:
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [ ]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


# Training Examples


In [ ]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [ ]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [ ]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Building Model

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                            ),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

# Loss Func

In [ ]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [ ]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 3.54651408e-03  1.54193572e-03  1.19518558e-03 ... -2.46529467e-04
    2.26686694e-04  3.22777103e-03]
  [ 2.41625425e-03  2.75857281e-03 -1.06388668e-03 ...  1.86652504e-03
   -2.60631088e-03  6.72125840e-04]
  [-1.51333792e-04  3.77886230e-03 -8.17798718e-04 ...  2.91840383e-03
   -2.98248674e-03 -2.85943737e-03]
  ...
  [-5.85086411e-03  8.90022889e-03 -5.18945977e-04 ...  3.58937454e-04
   -3.37513094e-03  4.25132457e-03]
  [-3.82212456e-03  7.61138648e-03  6.78765308e-03 ...  2.17179488e-03
   -3.91298300e-03  7.46537745e-03]
  [-5.19871106e-03  1.60845066e-03  7.08152261e-03 ...  3.52965691e-03
    9.19601298e-04  1.07164085e-02]]

 [[-8.14792176e-04 -2.46087392e-03  5.51923364e-03 ...  1.88936421e-03
    3.92983388e-03 -3.48628592e-03]
  [-5.92923164e-03  3.28129099e-05  1.59741484e-03 ... -9.41372477e-04
    1.55959267e-03 -3.30618327e-03]
  [-9.65873431e-03  2.17948924e-03 -9.92945395e-04 ... -2.57611671e-03
    3.37818637e-04 -3.38079291e-03]
  ...
  [-2.902

In [ ]:
pred = example_batch_predictions[0]
print(len(pred))

100


In [ ]:
time = pred[0]
print(len(time))

65


In [ ]:
time_pred = pred[0]
print(len(time_pred))

65


In [ ]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"!s!BZi&snBAEAfJTOyHDLbzxQvpQTDgAAZBjou:\nu-lAjPazfhhpcT.EzMAo& ;vNn!ft?ey':FRfsXljiPXUfi$YAkzq-:'.'X;"

In [ ]:
@keras.saving.register_keras_serializable(package="my_custom_package")
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

# Training

In [ ]:
history = model.fit(data, epochs=4, callbacks=[checkpoint_callback])

Epoch 1/4
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 73ms/step - loss: 2.0077
Epoch 2/4
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s 74ms/step - loss: 1.6503
Epoch 3/4
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 74ms/step - loss: 1.4958
Epoch 4/4
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 73ms/step - loss: 1.4132


In [ ]:
def generate_text(model, start_string):

  num_generate = 800


  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []


  temperature = 1.0

  # Here batch size == 1
  for i in range(num_generate):
      predictions = model(input_eval)

      predictions = predictions[0]


      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: Night
Night Turks from Lie'd tender dield
With God and gentlemen, whach knoss thy love as if
it dainy and confemence.

LUCIO:
Woo alone;
Which hath be or but into the insterrips?
Awedest me, to your words cannot he!
there are it more than his are maids of more love.

Second Tomfort.

TLANDIO:
I'll go, lorgs intermonity. I will not meet
By love forly as will have yet might behoved for you no merry with sight-hours
and fords and woe, help's tritake, let's the friends,
Sidos, means: for look'd like the bury me'n.

SEBASTIAN:
I swear thee, way tell fortunus and commints
Sift it tears. But you, night like a cursfail,
That's a traitor him mojes of the mal old ewn loves, bere advertion'd.
Though not yet shall be condinted host wan herselves.

VOLUMNIA:
Well were you.'

LEONTES:
This, and lets, then meancour
